In [ ]:
import rosbag2_py
import numpy as np
from scipy.io import savemat
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message

def read_rosbag(db3_file_path, topic_name):
    storage_options = rosbag2_py.StorageOptions(uri=db3_file_path, storage_id='sqlite3')
    converter_options = rosbag2_py.ConverterOptions('', '')
    reader = rosbag2_py.SequentialReader()
    reader.open(storage_options, converter_options)
    type_name = reader.get_all_topics_and_types()[0].type
    msg_type = get_message(type_name)
    imu_data = {'angular_velocity': [], 'linear_acceleration': [], 'magnetic_field': [], 'timestamps': []}

    while reader.has_next():
        (topic, data, timestamp) = reader.read_next()
        if topic == topic_name:
            msg = deserialize_message(data, msg_type)
            # Access nested imu and mag_field data
            imu_data['angular_velocity'].append([msg.imu.angular_velocity.x, msg.imu.angular_velocity.y, msg.imu.angular_velocity.z])
            imu_data['linear_acceleration'].append([msg.imu.linear_acceleration.x, msg.imu.linear_acceleration.y, msg.imu.linear_acceleration.z])
            imu_data['magnetic_field'].append([msg.mag_field.magnetic_field.x, msg.mag_field.magnetic_field.y, msg.mag_field.magnetic_field.z])
            imu_data['timestamps'].append(timestamp * 1e-9)

    imu_data['angular_velocity'] = np.array(imu_data['angular_velocity'])
    imu_data['linear_acceleration'] = np.array(imu_data['linear_acceleration'])
    imu_data['magnetic_field'] = np.array(imu_data['magnetic_field'])
    imu_data['timestamps'] = np.array(imu_data['timestamps'])

    return imu_data

# Replace with the actual path and topic
imu_data = read_rosbag('/home/rgautham20/lab3/fivehour1data.bag/fivehour1data.bag_0.db3', '/imu')

# Save to .mat file
savemat('imu_data.mat', imu_data)


[INFO] [1728940277.568765358] [rosbag2_storage]: Opened database '/home/rgautham20/lab3/fivehour1data.bag/fivehour1data.bag_0.db3' for READ_ONLY.
